In [1]:
import os
import pickle

from dotenv import load_dotenv
from pydantic import BaseModel

from src.coop import AsyncHandler, AsyncSeqLikeHandler, async_
from src.eff import Handler, Operation, coroutine_decorator
from src.llm import (
	AsyncOneRoundChatHandler,
	AsyncReplayLLMHandler,
	LLMHandler,
	OneRoundChatHandler,
	ReplayLLMHandler,
	TraceLLMHandler,
	complete,
	parse,
)
from src.util import Timer, awaitable_args_decorator

In [2]:
# TODO 注意这个！！！
import nest_asyncio
nest_asyncio.apply()

In [ ]:
load_dotenv()

#BASE_URL = os.getenv("BASE_URL")
#API_KEY = os.getenv("API_KEY")

#TODO lazy way!! will change!!!
BASE_URL='https://api.openai-proxy.org/v1'
API_KEY=''

MODEL = "gpt-5-nano"
TEMPERATURE = 1.0
MAX_TOKENS = 1000

In [20]:
import pandas as pd
from openai import OpenAI
import json
import time


In [21]:
client = OpenAI(
        base_url=BASE_URL,
        api_key=API_KEY,
    )

In [66]:
#df = pd.read_excel("subsidy.xlsx", nrows=1000)
df = pd.read_excel("subsidy.xlsx")

df.head()


,证券代码,证券简称,统计截止日期,列报会计科目,报表类型,项目,本期计入当期损益的金额,上期计入当期损益的金额,与资产相关/与收益相关
0,6,深振业A,2015-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,266250.0,210000.0,NaN
1,6,深振业A,2015-12-31,2,1,长沙市岳麓区财政局税收奖励金,1660000.0,NaN,NaN
2,6,深振业A,2015-12-31,2,1,西安浐灞生态区生态宜居示范单位奖励资金,150000.0,NaN,NaN
3,6,深振业A,2015-12-31,2,1,合计,2076250.0,210000.0,NaN
4,6,深振业A,2016-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,NaN,266250.0,NaN


In [23]:
df.shape

(1000, 9)

In [68]:
df = df[~df['项目'].astype(str).str.contains('合计', na=False)]
df.shape

(654054, 9)

In [69]:
len(pd.unique(df['项目']))

330681

In [57]:
df_short = df[0:10]

In [26]:
df_short.shape

(10, 9)

In [27]:
PROMPT_CLASSIFY = '''你是一名企业财务分析助理，任务是根据补贴或奖励项目名称，判断其属于以下补贴类型之一，并说明理由。

请严格按照以下分类标准执行：

1. 税收优惠和奖励：与退税、税收减免、税费返还、税收奖励等相关。例如：“税收奖励金”“减免税款”“税收返还”“即征即退”。
2. 研发补贴：与生产技术改造、新技术研发、创新资助、技术应用推广、产业化、专利相关。例如：“科技三项经费”“专利资助”“创新项目支持资金”。
3. 吸纳人才补贴：与引进人才、研究人员、科研团队、博士后、人才培养相关。例如：“博士后科研资助”“人才引进补贴”“科研人员经费”。
4. 产业升级补贴：与工业发展、产业扶持、生产设备更新、产业转型升级相关。例如：“产业发展奖励资金”“生产线改造补贴”“工业转型扶持资金”。
5. 项目运营补贴：与企业日常生产经营、土地整理、安全生产、电费、招商引资等有关。例如：“安全生产补贴”“招商引资奖励”“土地补偿资金”。
6. 就业补贴：与稳岗、就业、技能培训、职工补贴等有关。例如：“稳岗补贴”“就业奖励金”“技能培训补贴”。
7. 贸易补贴：与进出口业务、外贸发展、对外合作、市场拓展相关。例如：“外贸高质量发展资金”“出口奖励资金”“贸易促进基金”。
8. 环境保护和治理补贴：与节能环保、清洁生产、绿色发展、污染治理相关。例如：“节能专项资金”“环保设备改造补贴”“绿色生产补助”。
9. 信贷补贴：与贷款贴息、融资支持、信贷奖励有关。例如：“贷款贴息”“融资补助”“银行贷款利息补贴”。
10. 其他：不属于以上任何一类的项目。

请输出严格的 JSON 格式结果（不要包含额外文字或说明）：

{
  "project": "",
  "judgement_result": "",  // 从上面10类中选择一个类型
  "reason": ""         // 简要说明分类依据（不超过30字）
}

项目名称：
'''

In [28]:
def my_rule(project_name):
    """
    调用大模型，根据项目名称返回判断结果与理由。
    假设模型返回 JSON 格式字符串，例如：
    {"判断结果": "税收相关", "reason": "名称中含有‘税’字"}
    """
    # try:
        # 构造输入（这里你之后可以自定义 prompt）
    if project_name == "合计":
        return {
            "judgement_result": "其它",
            "reason": "合计"
        }

        
    prompt = PROMPT_CLASSIFY + project_name

    # 调用大模型（用最便宜的）
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "你是一个财务分类助手。"},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
    )

    # 提取返回文本
    content = response.choices[0].message.content.strip()


    # 解析JSON
    result = json.loads(content)

    # 返回一个 dict 以便多列合并
    return {
        "判断结果": result.get("judgement_result", "未知"),
        "reason": result.get("reason", "未提供理由")
    }

    # except Exception as e:
    #     # 错误处理（避免 apply 出错中断）
    #     return {
    #         "判断结果": "调用出错",
    #         "reason": str(e)
    #     }



In [29]:
start_time = time.time()  
df_short = df_short.join(df_short['项目'].apply(lambda x: pd.Series(my_rule(x))))
end_time = time.time() 
print(f"耗时：{end_time - start_time:.2f} 秒")


耗时：47.37 秒


In [30]:
df_short.head()

,证券代码,证券简称,统计截止日期,列报会计科目,报表类型,项目,本期计入当期损益的金额,上期计入当期损益的金额,与资产相关/与收益相关,判断结果,reason
0,6,深振业A,2015-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,266250.0,210000.0,NaN,税收优惠和奖励,涉及税收奖励金，属税收激励范畴
1,6,深振业A,2015-12-31,2,1,长沙市岳麓区财政局税收奖励金,1660000.0,NaN,NaN,税收优惠和奖励,属于税收奖励，税收优惠类
2,6,深振业A,2015-12-31,2,1,西安浐灞生态区生态宜居示范单位奖励资金,150000.0,NaN,NaN,环境保护和治理补贴,对生态宜居示范单位的奖励，促进环境治理
4,6,深振业A,2016-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,NaN,266250.0,NaN,税收优惠和奖励,与税收奖励相关
5,6,深振业A,2016-12-31,2,1,长沙市岳麓区财政局税收奖励金,NaN,1660000.0,NaN,税收优惠和奖励,与税收奖励相关


# now we need a handler version

In [31]:
classify = Operation()
log = Operation()
postprocess = Operation()

In [ ]:
class RuleSchema(BaseModel):
    project: str
    judgement_result: str
    reason: str


In [54]:
class FormatHandler(Handler):
    """
    将 RuleClassifyHandler 的输出结果格式化成规范 DataFrame 格式。
    处理的事件：format_result
    """

    def __init__(self):
        super().__init__()
        self.register(postprocess, self.postprocess)

    def postprocess(self, df_short, results):
        """
        df_short: 原始 DataFrame（含 '项目' 等列）
        results: classify 的结果列表（每个是 dict）
        """
        # 将 results 转成 DataFrame
        df_res = pd.DataFrame([x.model_dump() for x in results])
        # 规范列名
        rename_map = {
            "judgement_result": "判断结果",
            "reason": "理由"
        }
        df_res = df_res.rename(columns=rename_map)

        # 合并
        df_final = pd.concat([df_short.reset_index(drop=True), df_res], axis=1)

        # 整理列顺序（可选）
        # col_order = [c for c in df_short.columns] + ["判断结果", "理由"]
        # df_final = df_final[col_order]

        return df_final

In [41]:
class RuleClassifyHandler(Handler):
	"""
	handles: get_topics, get_description, log

	forward: parse, complete
	"""

	def __init__(self):
		super().__init__()
		self.register(classify, self.classify)
		self.register(log, print)

	def classify(self, project_name):
		response = parse("classify", PROMPT_CLASSIFY + project_name, RuleSchema)
		return response.choices[0].message.parsed


In [64]:
def main(df_short):
    results = []

    for project_name in df_short["项目"]:
        log(f"正在处理：{project_name}")
        result = classify(project_name)
        log(result)
        results.append(result)

    df_final = postprocess(df_short, results)
    return df_final


# 没有postprocess的版本

In [65]:
if __name__ == "__main__":
    with (
        LLMHandler(base_url=BASE_URL, api_key=API_KEY),
        TraceLLMHandler() as trace,
        OneRoundChatHandler(model="gpt-5-nano", temperature=1),
        RuleClassifyHandler(),
        FormatHandler()
    ):
        df_final = main(df_short)

    with open("rule_classify.trace", "wb") as f:
        pickle.dump(trace, f)


正在处理：长沙市岳麓区天顶街道办事处税收奖励金
judgement_result='税收优惠和奖励' reason='与税收奖励相关'
正在处理：长沙市岳麓区财政局税收奖励金
judgement_result='税收优惠和奖励' reason='税收奖励性质的财政补贴'
正在处理：西安浐灞生态区生态宜居示范单位奖励资金
judgement_result='环境保护和治理补贴' reason='与生态宜居示范相关的奖励资金'
正在处理：长沙市岳麓区天顶街道办事处税收奖励金
judgement_result='税收优惠和奖励' reason='涉及税收奖励金，属于税收优惠相关'
正在处理：长沙市岳麓区财政局税收奖励金
judgement_result='税收优惠和奖励' reason='与税收奖励相关'
正在处理：西安浐灞生态区生态宜居示范单位奖励资金
judgement_result='环境保护和治理补贴' reason='与环保与绿色发展相关的奖励资金'
正在处理：西安浐灞生态区管理委员会2015年度突出贡献企业奖励资金
judgement_result='其他' reason='政府对突出贡献企业的现金奖励，非特定类别'
正在处理：西安浐灞生态区管理委员会2015年度稳增长促投资优秀企业奖励资金
judgement_result='项目运营补贴' reason='含招商引资奖励，属项目运营补贴'
正在处理：长沙市住房和城乡建设委员会长沙可再生能源建筑应用示范项目专项补助资金
judgement_result='环境保护和治理补贴' reason='支持可再生能源建筑示范，环保类'
正在处理：2016年青秀区房地产项目购房促销活动补贴
judgement_result='其他' reason='用途为购房促销，非九类明确分类'


In [59]:
df_final

,证券代码,证券简称,统计截止日期,列报会计科目,报表类型,项目,本期计入当期损益的金额,上期计入当期损益的金额,与资产相关/与收益相关,判断结果,理由
0,6,深振业A,2015-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,266250.0,210000.0,NaN,税收优惠和奖励,与税收奖励相关，归类为税收优惠与奖励
1,6,深振业A,2015-12-31,2,1,长沙市岳麓区财政局税收奖励金,1660000.0,NaN,NaN,税收优惠和奖励,名称含税收奖励金，归类为税收优惠和奖励
2,6,深振业A,2015-12-31,2,1,西安浐灞生态区生态宜居示范单位奖励资金,150000.0,NaN,NaN,环境保护和治理补贴,与环境保护和治理相关的补贴
3,6,深振业A,2016-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,NaN,266250.0,NaN,税收优惠和奖励,含税收奖励金，属于税收优惠与奖励
4,6,深振业A,2016-12-31,2,1,长沙市岳麓区财政局税收奖励金,NaN,1660000.0,NaN,税收优惠和奖励,与税收奖励相关
5,6,深振业A,2016-12-31,2,1,西安浐灞生态区生态宜居示范单位奖励资金,NaN,150000.0,NaN,环境保护和治理补贴,用于生态宜居示范奖励，属环保治理补贴
6,6,深振业A,2016-12-31,2,1,西安浐灞生态区管理委员会2015年度突出贡献企业奖励资金,300000.0,NaN,NaN,项目运营补贴,对企业经营的奖励资金，属运营补贴范畴
7,6,深振业A,2016-12-31,2,1,西安浐灞生态区管理委员会2015年度稳增长促投资优秀企业奖励资金,10000.0,NaN,NaN,项目运营补贴,招商引资相关的运营补贴
8,6,深振业A,2016-12-31,2,1,长沙市住房和城乡建设委员会长沙可再生能源建筑应用示范项目专项补助资金,62400.0,NaN,NaN,环境保护和治理补贴,绿色建筑示范，环保治理导向
9,6,深振业A,2016-12-31,2,1,2016年青秀区房地产项目购房促销活动补贴,50000.0,NaN,NaN,项目运营补贴,用于房地产项目促销的运营性补贴


# 有后处理+缓存的版本

In [62]:
with open("rule_classify.trace", "rb") as f:
	trace = pickle.load(f)
with Timer() as t_sync:
    with (
        ReplayLLMHandler(trace,no_sleep=True),
        OneRoundChatHandler(model="gpt-5-nano", temperature=1),
        RuleClassifyHandler(),
        FormatHandler()
    ):
        df_final = main(df_short)



正在处理：长沙市岳麓区天顶街道办事处税收奖励金
judgement_result='税收优惠和奖励' reason='与税收奖励相关，归类为税收优惠与奖励'
正在处理：长沙市岳麓区财政局税收奖励金
judgement_result='税收优惠和奖励' reason='名称含税收奖励金，归类为税收优惠和奖励'
正在处理：西安浐灞生态区生态宜居示范单位奖励资金
judgement_result='环境保护和治理补贴' reason='与环境保护和治理相关的补贴'
正在处理：长沙市岳麓区天顶街道办事处税收奖励金
judgement_result='税收优惠和奖励' reason='含税收奖励金，属于税收优惠与奖励'
正在处理：长沙市岳麓区财政局税收奖励金
judgement_result='税收优惠和奖励' reason='与税收奖励相关'
正在处理：西安浐灞生态区生态宜居示范单位奖励资金
judgement_result='环境保护和治理补贴' reason='用于生态宜居示范奖励，属环保治理补贴'
正在处理：西安浐灞生态区管理委员会2015年度突出贡献企业奖励资金
judgement_result='项目运营补贴' reason='对企业经营的奖励资金，属运营补贴范畴'
正在处理：西安浐灞生态区管理委员会2015年度稳增长促投资优秀企业奖励资金
judgement_result='项目运营补贴' reason='招商引资相关的运营补贴'
正在处理：长沙市住房和城乡建设委员会长沙可再生能源建筑应用示范项目专项补助资金
judgement_result='环境保护和治理补贴' reason='绿色建筑示范，环保治理导向'
正在处理：2016年青秀区房地产项目购房促销活动补贴
judgement_result='项目运营补贴' reason='用于房地产项目促销的运营性补贴'
-0--------
[RuleSchema(judgement_result='税收优惠和奖励', reason='与税收奖励相关，归类为税收优惠与奖励'), RuleSchema(judgement_result='税收优惠和奖励', reason='名称含税收奖励金，归类为税收优惠和奖励'), RuleSchema(judgement_result='环境保护和治理补贴', reason='与环

In [63]:
df_final

,证券代码,证券简称,统计截止日期,列报会计科目,报表类型,项目,本期计入当期损益的金额,上期计入当期损益的金额,与资产相关/与收益相关,判断结果,理由
0,6,深振业A,2015-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,266250.0,210000.0,NaN,税收优惠和奖励,与税收奖励相关，归类为税收优惠与奖励
1,6,深振业A,2015-12-31,2,1,长沙市岳麓区财政局税收奖励金,1660000.0,NaN,NaN,税收优惠和奖励,名称含税收奖励金，归类为税收优惠和奖励
2,6,深振业A,2015-12-31,2,1,西安浐灞生态区生态宜居示范单位奖励资金,150000.0,NaN,NaN,环境保护和治理补贴,与环境保护和治理相关的补贴
3,6,深振业A,2016-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,NaN,266250.0,NaN,税收优惠和奖励,含税收奖励金，属于税收优惠与奖励
4,6,深振业A,2016-12-31,2,1,长沙市岳麓区财政局税收奖励金,NaN,1660000.0,NaN,税收优惠和奖励,与税收奖励相关
5,6,深振业A,2016-12-31,2,1,西安浐灞生态区生态宜居示范单位奖励资金,NaN,150000.0,NaN,环境保护和治理补贴,用于生态宜居示范奖励，属环保治理补贴
6,6,深振业A,2016-12-31,2,1,西安浐灞生态区管理委员会2015年度突出贡献企业奖励资金,300000.0,NaN,NaN,项目运营补贴,对企业经营的奖励资金，属运营补贴范畴
7,6,深振业A,2016-12-31,2,1,西安浐灞生态区管理委员会2015年度稳增长促投资优秀企业奖励资金,10000.0,NaN,NaN,项目运营补贴,招商引资相关的运营补贴
8,6,深振业A,2016-12-31,2,1,长沙市住房和城乡建设委员会长沙可再生能源建筑应用示范项目专项补助资金,62400.0,NaN,NaN,环境保护和治理补贴,绿色建筑示范，环保治理导向
9,6,深振业A,2016-12-31,2,1,2016年青秀区房地产项目购房促销活动补贴,50000.0,NaN,NaN,项目运营补贴,用于房地产项目促销的运营性补贴


# 在线缓存版本